## eCommerce Analytics – case study

Loading the data using 'pandas'.

In [ ]:
import os
print("Current working directory:", os.getcwd())


# import pandas as pd

# db_path = 'data/eCommerce API dataset.csv'
# df = pd.read_csv(db_path)
# df.head()

Hello!
